In [1]:
# Dependencies
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv("../db/nutrition.csv")
# df = pd.read_csv("nutrition.csv", index_col ="NDB_No")

In [3]:
df.head()

,NDB_No,Shrt_Desc,Water,Energy,Protein,Lipid_Total,Carbohydrate,Fiber,Sugar_Total,Calcium,...,Vitamin_K,FA_Sat_g,FA_Mono_g,FA_Poly_g,Cholestrol,Weight_grams,Weight_desc,GmWt_2,GmWt_Desc2,Refuse_Pct
0,1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,0.06,0.0,0.06,24,...,7.0,51.368,21.021,3.043,215,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0
1,1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,2.87,0.0,0.06,23,...,4.6,45.390,19.874,3.331,225,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0
2,1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.0,0.00,4,...,8.6,61.924,28.732,3.694,256,12.80,1 tbsp,205.0,1 cup,0
3,1004,"CHEESE,BLUE",42.41,353,21.40,28.74,2.34,0.0,0.50,528,...,2.4,18.669,7.778,0.800,75,28.35,1 oz,17.0,1 cubic inch,0
4,1005,"CHEESE,BRICK",41.11,371,23.24,29.68,2.79,0.0,0.51,674,...,2.5,18.764,8.598,0.784,94,132.00,"1 cup, diced",113.0,"1 cup, shredded",0


In [4]:
df.columns

Index(['NDB_No', 'Shrt_Desc', 'Water', 'Energy', 'Protein', 'Lipid_Total',
       'Carbohydrate', 'Fiber', 'Sugar_Total', 'Calcium', 'Iron', 'Magnesium',
       'Phosphorus', 'Potassium', 'Sodium', 'Zinc', 'Copper', 'Manganese',
       'Selenium', 'Vitamin_C', 'Thiamin', 'Riboflavin', 'Niacin',
       'Panto_Acid', 'Vitamin_B6', 'Folate_Total', 'Folic_Acid',
       'Food_Folate_mcg', 'Folate_DFE_mcg', 'Choline_Tot_mg', 'Vitamin_B12',
       'Vit_A_IU', 'Vitamin_A', 'Retinol', 'Alpha_Carot_mcg', 'Beta_Carot_mcg',
       'Beta_Crypt_mcg', 'Lycopene_mcg', 'Lut_Zea_mcg', 'Vitamin_E',
       'Vitamin_D', 'Vit_D_IU', 'Vitamin_K', 'FA_Sat_g', 'FA_Mono_g',
       'FA_Poly_g', 'Cholestrol', 'Weight_grams', 'Weight_desc', 'GmWt_2',
       'GmWt_Desc2', 'Refuse_Pct'],
      dtype='object')

# Change the columns to per calorie value

In [5]:
df["Protein/cal"] = df["Protein"] / df["Energy"]
df["Carbohydrtes/cal"] = df["Carbohydrate"] / df["Energy"]
df["Sodium/cal"] = df["Sodium"]/ df["Energy"]
df["Cholestrol/cal"] = df["Cholestrol"]/ df["Energy"]
df["Sugar/cal"] = df["Sugar_Total"]/ df["Energy"]
df["Calcium/cal"] = df["Calcium"]/ df["Energy"]

In [6]:
df_percalorie = df[["Shrt_Desc", "Protein/cal", "Carbohydrtes/cal", "Sodium/cal", "Cholestrol/cal", "Sugar/cal", "Calcium/cal"]]

In [7]:
df_percalorie.head()

,Shrt_Desc,Protein/cal,Carbohydrtes/cal,Sodium/cal,Cholestrol/cal,Sugar/cal,Calcium/cal
0,"BUTTER,WITH SALT",0.001185,0.000084,0.896792,0.299861,0.000084,0.033473
1,"BUTTER,WHIPPED,W/ SALT",0.000682,0.003997,0.811978,0.313370,0.000084,0.032033
2,"BUTTER OIL,ANHYDROUS",0.000320,0.000000,0.002283,0.292237,0.000000,0.004566
3,"CHEESE,BLUE",0.060623,0.006629,3.246459,0.212465,0.001416,1.495751
4,"CHEESE,BRICK",0.062642,0.007520,1.509434,0.253369,0.001375,1.816712


## Removing null values from dataframe

In [8]:
df_percalorie.isnull().sum()

Shrt_Desc            0
Protein/cal         36
Carbohydrtes/cal    31
Sodium/cal           9
Cholestrol/cal      39
Sugar/cal           38
Calcium/cal         13
dtype: int64

In [9]:
df_percalorie = df_percalorie.dropna(how='any',axis=0)

In [10]:
df_percalorie.isnull().sum()

Shrt_Desc           0
Protein/cal         0
Carbohydrtes/cal    0
Sodium/cal          0
Cholestrol/cal      0
Sugar/cal           0
Calcium/cal         0
dtype: int64

In [11]:
df_percalorie.shape

(8751, 7)

## Cosine similarity

In [12]:
df_percalorie.columns

Index(['Shrt_Desc', 'Protein/cal', 'Carbohydrtes/cal', 'Sodium/cal',
       'Cholestrol/cal', 'Sugar/cal', 'Calcium/cal'],
      dtype='object')

In [13]:
X_nut = df_percalorie[['Protein/cal', 'Carbohydrtes/cal', 'Sodium/cal', 'Cholestrol/cal', 'Sugar/cal', 'Calcium/cal']].values

In [14]:
X_norm = Normalizer().fit_transform(X_nut)

In [15]:
# Find the index for search item
print(int(df_percalorie[df_percalorie['Shrt_Desc']=='SOY PROT ISOLATE,K TYPE'].index.values))

5009


## Display list of 10 similar items

In [16]:
# term = 'SOY PROT ISOLATE,K TYPE'

In [17]:
# idx = int(df_percalorie[df_percalorie['Shrt_Desc']== term].index.values)

In [18]:
# X_norm[5009].reshape(1,-1)

In [19]:
# X_norm[idx].reshape(1,-1)

In [20]:
# similarities = cosine_similarity(X_norm[idx].reshape(1,-1), X_norm)
# k = 10
# result = np.sort(np.argpartition(similarities[0], len(similarities[0]) - k)[-k:])

# #idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
# df_percalorie.iloc[result]

In [21]:
# Recommend 5 items similar to text in search string
def similar_items(term):
    idx = int(df_percalorie[df_percalorie['Shrt_Desc']== term].index.values)
    simularities = cosine_similarity(X_norm[idx].reshape(1,-1), X_norm)
    k = 10
    result = np.sort(np.argpartition(simularities[0], len(simularities[0]) - k)[-k:])

    #idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
    return df_percalorie.iloc[result]
if __name__ == "__main__":
    term = "BUTTER,WITH SALT"
    term1 = "choclate cooies mnt"
    print(similar_items(term)["Shrt_Desc"])

0                                        BUTTER,WITH SALT
1                                  BUTTER,WHIPPED,W/ SALT
141                          EGG,YOLK,RAW,FRZ,SALTED,PAST
1068    CHICKEN,BROILERS OR FRYERS,SKN ONLY,CKD,ROTISS...
1073    CHICKEN,BROILERS OR FRYERS,DRUMSTK,MEAT & SKN,...
1081    CHICKEN,BROILER,ROTISSERIE,BBQ,DRUMSTK MEAT & SKN
1082      CHICKEN,BROILER,ROTISSERIE,BBQ,THIGH MEAT & SKN
1663                         PATE,LIVER,NOT SPECIFIED,CND
5743                    CREAM PUFF SHELL,PREP FROM RECIPE
6854                                MCDONALD'S,BIG BRKFST
Name: Shrt_Desc, dtype: object
